## An Overview

Auto-completion functionalities are now ubiquitous in search engines, document editors, and messaging apps. Here, I am developing an algorithmic strategy from scratch to implement these computational solutions. 

A [trie tree](https://en.wikipedia.org/wiki/Trie), or a prefix tree, is a common data structure that stores a set of strings in a collection of nodes so that all strings with a common prefix are found in the same branch of the tree. Each node is associated with a letter, and as you traverse down the tree, you pick up more letters, eventually forming a word. Complete words are commonly found on the leaf nodes. However, some inner nodes can also mark full words.

Let’s use an example diagram to illustrate several important features of tries:

In [2]:
from IPython.core.display import Image, display
display(Image(url='https://drive.google.com/uc?id=1NxHPsTzU3xEz2Ivck1NyqKpOix4Sc5iE', width=200))

C:\Users\kresh\AppData\Local\Temp\ipykernel_20608\3475477571.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import Image, display


Here are a few things to note from the schematics above:

- Nodes that mark valid words are marked in yellow. Notice that while all leaves are considered valid words, only some inner nodes contain valid words, while some remain only prefixes to valid words appearing down the branch.

- The tree does not have to be balanced, and the height of different branches depends on its contents.

- In our implementation, branches never merge to show common suffixes (for example, both ANT and ART end in T, but these nodes are kept separate in their respective branches). However, this is a standard first line of memory optimization for tries.

- The first node contains an empty string; it “holds the tree together.”

## Python implementation of a trie tree

### Theoretical pondering
<b> What is the better approach: making separate **Tree and Node** classes, or only making a **Node** class? </b>

Generally, it is more convenient to handle trees implemented using two classes. In this case, one will be able to handle unusual cases more efficiently. For example, if one had an empty tree and there are two separate classes for the tree and nodes, they could still use the methods set by the tree class. If there was only one class, it would get stuck because all the methods would have to work within the non-existent node.

Besides, using two different classes, one can more easily dynamically create new nodes and insert them into the tree. Using only one class would be much less convenient because each time, they'd use the same one class that has a lot more methods embedded beside the assignment of the attributes. It would be harder to update the tree with new values using the old structure. Besides, having a separate class for the nodes allows us easily create nodes for multiple different trees.

### Practical implementation

In [3]:
class Node:
    """
    This class represents one node of a trie tree.
    """

    def __init__(self, char):
        
        #assigning value to the node and creating space for its kids
        self.char = char
        self.children = []
        
        #indicator of whether the character is the end of a valid word
        self.is_end = False
        
        
class Trie:
    """
    This class represents the entirety of a trie tree.

    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    """
    
    def __init__(self, word_list = None):
        """
        Creates the Trie instance, inserts initial words if provided.
        
        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
        
        #setting root to an emty string        
        self.root = Node("")
        
        #converting all the words in the input list to lowercase 
        ##and inserting them into the tree
        if word_list:
            for word in word_list:
                word = word.lower()
                self.insert(word)
        
    
    def insert(self, word):
        """
        Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.
        """
        
        #comparing the characters in the word with nodes starting from the root
        node = self.root 
        
        #looking for the characters in the tree
        for char in word:
            found_in_child = False
            for child_node in node.children:
                if child_node.char == char:
                    node = child_node
                    found_in_child = True
        
            #if the character is not in the tree yet placing it after the letters in the word 
            ##that are already present in the tree if any
            if not found_in_child:
                new_node = Node(char)
                node.children.append(new_node)
                node = new_node
        
        #marking the end of the word
        node.is_end = True

        
    def lookup(self, word):
        """
        Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
        """

        node = self.root
        
        #if tree is empty returning False right away
        if not node.children:
            return False
        
        #converting the word we are looking for to lowercase as well
        word = word.lower()
            
            
        #looking for all the characters in the word that would be places consequently
        for char in word:
            char_not_found = True
            for child in node.children:
                if child.char == char:
                    char_not_found = False
                    node = child
                    break
        
            if char_not_found:
                return False
        
        #telling whether the full word is present in the tree
        return node.is_end

In [4]:
# This is Namárië, JRRT's elvish poem written in Quenya
wordbank = "Ai! laurië lantar lassi súrinen, yéni unótimë ve rámar aldaron! \
Yéni ve lintë yuldar avánier mi oromardi lisse-miruvóreva Andúnë pella, \
Vardo tellumar nu luini yassen tintilar i eleni ómaryo airetári-lírinen. \
Sí man i yulma nin enquantuva? An sí Tintallë Varda Oiolossëo ve fanyar máryat Elentári ortanë, \
ar ilyë tier undulávë lumbulë; ar sindanóriello caita mornië i falmalinnar imbë met, \
ar hísië untúpa Calaciryo míri oialë. Sí vanwa ná, Rómello vanwa, Valimar! Namárië! \
Nai hiruvalyë Valimar. Nai elyë hiruva. \
Namárië!".replace("!", "").replace("?", "").replace(".", "").replace(",", "").replace(";", "").split()

trie = Trie(wordbank)

assert trie.lookup('oiolossëo') == True
# this is a prefix, but also a word in itself
assert trie.lookup('An') == True
# this is a prefix, but NOT a word
assert trie.lookup('ele') == False
# not in the wordbank
assert trie.lookup('Mithrandir') == False

So far, we see that the code easily finds English words present in the text regardless of whether they are lower or uppercase. I'll check how the code manages texts in different languages. I'm looking for the Ukrainian word "Hello" in Cyrillic. If it doesn't return any errors and just doesn't find it, we are all good. That would mean that different alphabets do not break the code.

Also, I am checking the same property for numbers here. 

In [5]:
assert trie.lookup('Привіт') == False
assert trie.lookup('233') == False

Yay! As expected, the code doesn't return any errors and just doesn't find the numbers and words that are not present in the text. 

In the code cell below, I am checking whether the code can really work with different alphabets. I input "HeLlo, my family arrives on March 17th" in Ukrainian. I am using both numbers and the Ukrainian language in the sentence to check how universal my program is. As we can see, it can indeed search and find numbers, words in different languages and with random uppercase letters, even the words that have special characters (word family "сім'я" in Ukrainian.)

In [6]:
wordbank2 = "Привіт, моя сім'я приїжджає 17 березня".replace("!", "").replace("?", "").replace(".", "").replace(",", "").replace(";", "").split()
trie2 = Trie(wordbank2)

assert trie2.lookup('ПриВіт') == True
assert trie2.lookup("сім'я") == True

In [7]:
assert trie2.lookup('17') == True

All the tests I could come up with on my own are passing. The only thing is that the user/customer should decide whether they want the code to detect words written in lowercase even though they were expected to be in uppercase. If they don't want to "accept" all the words, we should alter the code so it doesn't convert all the words to the lowercase. Instead, it returns an error if, for example, the word that is not right after a dot (a.k.a., is not at the beginning of the sentence) and starts with a capital letter/has any capital letters in it. But it is pretty complex since some words like names must always start with a capital letter. 

## The computational complexity of tries

The insert runtime for the trie tree depends on the length of the word we are inserting because we have to iterate through every letter, compare it to the existent nodes, and if the letter is not there yet, add it at the right spot. So, the complexity of insert() is O(N). 

Therefore, the average complexity of the trie tree constructions is N*L, where L is the average length of the words we are inserting, and N is the number of words. In the worst-case/best-case, L is constant for all words. Whether it is best or worst depends on the context. If the word is very long and we wish it was smaller because it would be easier to store, it is probably the worst-case scenario. 

lookup() has the same complexity of O(L) because to find the word, we are iterating through all the characters in the word again (using the for-loop) and comparing them to children on each level. The comparisons utilize for-loop as well; however, the number of children ranges a lot. We might have many nodes attached to one node having C = N-n, where n is the order of the parent node we are looking at. However, the remaining loops would not have that many nodes to look at. Eventually, it averages out back to O(L.) The best and worst-case scenarios are analogous to the ones in the insertion. 

BST stores comparable data, like numbers, because the placement of new nodes heavily depends on how larger/smaller is the new node as compared to the parent. In comparison, tries are only interested in whether the node is present within kids, and that's it. Because of that, it is tough to have a balanced trie. There must be nearly the same number of words with the same prefix. 

We can still adjust the BST, though, to make its keys be represented as strings. We could use the alphabetical order as the comparison variable in this case. Then potentially, we could create some sort of a dictionary BST, where each right child is located further in the alphabetical order than the left child as compared to the parent. 
 
Potentially BST would even be able to help us out with finding the words with the same prefixes. We'd have somehow to convert the string to its representation of alphabetical order and find the subtree that has this value as a parent. Some of the children of that node supposedly would have the same prefix. The insertion and search in BST with strings would be the same as for any other BST - O(h), where h is the tree's height. Since every node represents one word, the height would directly depend on the number of words and, on average, would equal nlogn. In the worst case, it would get up to O(n) if there's only one branch with the words. There would not be any different from the list of the evaluations of the alphabetical order of the given words. 

Since the complexity of operations in BST depends on the number of words but not their length, if we have a few very long words, BST might actually have a smaller runtime. 


However, if there are many words in the BST, there is a high probability that the children on lower levels connected to that parent would have different prefixes because they would get further in the alphabetical order. Trie tree guarantees us finding the words with a given prefix, though, so this data structure is valuable. There are no other data structures that would be able to support such operation (at least from the ones I can recall.)

## Finding the $k$ most common words in a speech.

In [8]:
class Node:
    """
    This class represents one node of a trie tree.
    """

    def __init__(self, char):
        
        self.char = char
        self.children = []
        self.is_end = False
        
        #counting how many times the word appeared
        self.counter = 0
        
        
class Trie:
    """
    This class represents the entirety of a trie tree.
    
    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    """
    
    def __init__(self, word_list = None):
        """Creates the Trie instance, inserts initial words if provided.
        
        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
           
        self.root = Node("")
    
        #creating a dict to track the words and the number of times they appeared
        self.word_count = {}
        
        if word_list:
            for word in word_list:
                word = word.lower()
                self.insert(word)
        
    
    def insert(self, word):
        """
        Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.
        """
        
        node = self.root 
        
        for char in word:
            found_in_child = False
            for child_node in node.children:
                if child_node.char == char:
                    node = child_node
                    found_in_child = True
        
            if not found_in_child:
                new_node = Node(char)
                node.children.append(new_node)
                node = new_node
        
        node.is_end = True
        
        #recording that the word appeared 
        node.counter += 1
        
        #updating the counter for the word
        self.word_count[word] = node.counter

        
    def lookup(self, word):
        """
        Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
        """

        node = self.root
        
        if not node.children:
            return False
        
        word = word.lower()
            
            
        for char in word:
            char_not_found = True
            for child in node.children:
                if child.char == char:
                    char_not_found = False
                    node = child
                    break
        
            if char_not_found:
                return False
        
        return node.is_end
    
    
    def k_most_common(self, k):
        """
        Finds k words inserted into the trie most often.
        
        Parameters
        ----------
        k : int
            Number of most common words to be returned.

        Returns
        ----------
        list
            List of tuples.
            
            Each tuple entry consists of the word and its frequency.
            The entries are sorted by frequency.
        """
        #sorting the words and counts alphabetically and then by the count values
        sorted_count = {k: v for k, v in sorted(self.word_count.items(), key=lambda item: item[0])}
        sorted_count = {k: v for k, v in sorted(sorted_count.items(), key=lambda item: item[1], reverse = True)}

        lst_common = []
        
        #picking k the most common words and returning the corresponding tuples
        for key, value in sorted_count.items():
            lst_common.append((key, value))
            k -=1
            if k == 0:
                break
        return lst_common
        

In the code cell below, I am checking whether the code counts words right so far. 

In [9]:
trie = Trie(wordbank)

print(trie.word_count)

{'ai': 1, 'laurië': 1, 'lantar': 1, 'lassi': 1, 'súrinen': 1, 'yéni': 2, 'unótimë': 1, 've': 3, 'rámar': 1, 'aldaron': 1, 'lintë': 1, 'yuldar': 1, 'avánier': 1, 'mi': 1, 'oromardi': 1, 'lisse-miruvóreva': 1, 'andúnë': 1, 'pella': 1, 'vardo': 1, 'tellumar': 1, 'nu': 1, 'luini': 1, 'yassen': 1, 'tintilar': 1, 'i': 3, 'eleni': 1, 'ómaryo': 1, 'airetári-lírinen': 1, 'sí': 3, 'man': 1, 'yulma': 1, 'nin': 1, 'enquantuva': 1, 'an': 1, 'tintallë': 1, 'varda': 1, 'oiolossëo': 1, 'fanyar': 1, 'máryat': 1, 'elentári': 1, 'ortanë': 1, 'ar': 3, 'ilyë': 1, 'tier': 1, 'undulávë': 1, 'lumbulë': 1, 'sindanóriello': 1, 'caita': 1, 'mornië': 1, 'falmalinnar': 1, 'imbë': 1, 'met': 1, 'hísië': 1, 'untúpa': 1, 'calaciryo': 1, 'míri': 1, 'oialë': 1, 'vanwa': 2, 'ná': 1, 'rómello': 1, 'valimar': 2, 'namárië': 2, 'nai': 2, 'hiruvalyë': 1, 'elyë': 1, 'hiruva': 1}


We can count the words, so now I'll comу back to the code cell above and finish the sorting part and output k most common words. After adjustment, I am running the tests below. 

In [10]:
!pip install requests

from requests import get
speakers = ['Faruqi', 'Kennedy', 'King', 'Thunberg', 'Havel']
bad_chars = [';', ',', '.', '?', '!', '_', 
             '[', ']', ':', '“', '”', '"', '–', '-']

for speaker in speakers:
    
    # download and clean up the speech from extra characters
    speech_full = get(f'https://bit.ly/CS110-{speaker}').text
    just_text = ''.join(c for c in speech_full if c not in bad_chars)
    without_newlines = ''.join(c if (c not in ['\n', '\r', '\t']) else " " for c in just_text)
    just_words = [word for word in without_newlines.split(" ") if word != ""]
    
    trie = Trie(just_words)
    
    if speaker == 'Faruqi':
        Faruqi = [('the', 60), ('and', 45), ('to', 39), ('in', 37), 
                  ('of', 34), ('is', 25), ('that', 22), ('this', 21), 
                  ('a', 20), ('people', 20), ('has', 14), ('are', 13), 
                  ('for', 13), ('we', 13), ('have', 12), ('racism', 12), 
                  ('black', 11), ('justice', 9), ('lives', 9), ('police', 9)]
        assert trie.k_most_common(20) == Faruqi
    
    elif speaker == 'Kennedy':
        Kennedy = [('the', 117), ('and', 109), ('of', 93), ('to', 63), 
                   ('this', 44), ('in', 43), ('we', 43), ('a', 39), 
                   ('be', 30), ('for', 27), ('that', 27), ('as', 26), 
                   ('it', 24), ('will', 24), ('new', 22), ('space', 22), 
                   ('is', 21), ('all', 15), ('are', 15), ('have', 15), ('our', 15)]
        assert trie.k_most_common(21) == Kennedy
    
    elif speaker == 'Havel':
        Havel = [('the', 34), ('of', 23), ('and', 20), ('to', 15), 
                 ('in', 13), ('a', 12), ('that', 12), ('are', 9), 
                 ('we', 9), ('have', 8), ('human', 8), ('is', 8), 
                 ('you', 8), ('as', 7), ('for', 7), ('has', 7), ('this', 7), 
                 ('be', 6), ('it', 6), ('my', 6), ('our', 6), ('world', 6)]
        assert trie.k_most_common(22) == Havel
    
    elif speaker == 'King':
        King = [('the', 103), ('of', 99), ('to', 59), ('and', 54), ('a', 37), 
                ('be', 33), ('we', 29), ('will', 27), ('that', 24), ('is', 23), 
                ('in', 22), ('as', 20), ('freedom', 20), ('this', 20), 
                ('from', 18), ('have', 17), ('our', 17), ('with', 16), 
                ('i', 15), ('let', 13), ('negro', 13), ('not', 13), ('one', 13)]
        assert trie.k_most_common(23) == King
    
    elif speaker == 'Thunberg':
        Thunberg = [('you', 22), ('the', 20), ('and', 16), ('of', 15), 
                    ('to', 14), ('are', 10), ('is', 9), ('that', 9), 
                    ('be', 8), ('not', 7), ('with', 7), ('i', 6), 
                    ('in', 6), ('us', 6), ('a', 5), ('how', 5), ('on', 5), 
                    ('we', 5), ('all', 4), ('dare', 4), ('here', 4), 
                    ('my', 4), ('people', 4), ('will', 4)]
        assert trie.k_most_common(24) == Thunberg


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\kresh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Now, I'll check this code using the text provided before. That one has more varied characters. Besides, since it is a lot smaller text we can read it ourselves and track any mistakes on the spot. 

In [11]:
trie = Trie(wordbank)

assert trie.k_most_common(5) == [('ar', 3), ('i', 3), ('sí', 3), ('ve', 3), ('nai', 2)]

In the code cell below, I'll popularize the Ukrainian language a bit more. Here I am uploading a text file with a Ukrainian novel "Intermezzo" by Mykhailo Kotsubynskyi. This is a novel about a guy who got tired of the hustle in the city and went to a village to relax. But he constantly has flashbacks about people back in the city he cares about and, after taking a bit of breath in the village, returns to the city to help and be three for others. Really cute and empowering. 

This text is another material to work with and see whether my code actually works. Besides, it can be used to prove that my code can work with multiple languages, a.k.a. it is quite universal.

In [12]:
file = open('Intermezzo.txt', encoding="utf8")
text = file.read()
text = text.replace("!", "").replace("?", "").replace(".", "").replace(",", "").replace(";", "").replace("-", "").replace("*", "").replace("—", "").split()

trie = Trie(text)
trie.k_most_common(5)

[('і', 140), ('я', 86), ('на', 71), ('в', 70), ('як', 65)]

It turns out the most popular word in that novel is "and" in Ukrainian, which is the "i" in the first tuple. The rest of the top most popular words are mostly prepositions "on," "in," and the pronounce "I." This is very plausible because we constantly use these same words in combination with many others. For the third test, I'll look for more common words in that text.

In [13]:
trie.k_most_common(15)

[('і', 140),
 ('я', 86),
 ('на', 71),
 ('в', 70),
 ('як', 65),
 ('що', 52),
 ('не', 51),
 ('й', 49),
 ('з', 45),
 ('у', 42),
 ('а', 40),
 ('мене', 37),
 ('ти', 33),
 ('так', 25),
 ('до', 21)]

The output in the code cell above contained characters such as * and —; therefore, I added them to the list of characters that must be removed before building the tree. The test helped me improve my code a tiny bit! The output above is plausible as well because it is just prepositions and pronouns:

'і'    - 'and'         - 140,

'я'    - 'I'            - 86,

'на'   - 'on'           - 71,

'в'    - 'in'           - 70,

'як'   - 'as'           - 65,

'що'   - 'that'         - 52,

'не'   - 'not'          - 51,

'й'    - 'and'          - 49,

'з'    - 'with'         - 45,

'у'    - 'in'/'next to' - 42,

'а'    - 'and'/'but'    - 40,

'мене' - 'me'           - 37,

'ти'   - 'you'          - 33,

'так'  - 'this way'     - 25,

'до'   - 'to'           - 21


## Implementing an autocomplete with a Shakespearean dictionary!

In [14]:
class Node:
    """
    This class represents one node of a trie tree.
    """

    def __init__(self, char):
        
        self.char = char
        self.children = []
        self.is_end = False
        self.counter = 0
        
        
class Trie:
    """
    This class represents the entirety of a trie tree.
    
    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    """
    
    def __init__(self, word_list = None):
        """
        Creates the Trie instance, inserts initial words if provided.
        
        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
        
        self.root = Node("")
        self.word_count = {}
        
        if word_list:
            for word in word_list:
                word = word.lower()
                self.insert(word)
        
    
    def insert(self, word):
        """
        Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.
        """
        
        node = self.root 
        
        for char in word:
            found_in_child = False
            for child_node in node.children:
                if child_node.char == char:
                    node = child_node
                    found_in_child = True
        
            if not found_in_child:
                new_node = Node(char)
                node.children.append(new_node)
                node = new_node

        node.is_end = True
        
        node.counter += 1
        
        self.word_count[word] = node.counter

        
    def lookup(self, word):
        """
        Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
        """

        node = self.root
        
        if not node.children:
            return False
        
        word = word.lower()
            
        for char in word:
            char_not_found = True
            for child in node.children:
                if child.char == char:
                    char_not_found = False
                    node = child
                    break
        
            if char_not_found:
                return False
        
        return node.is_end
    
    def k_most_common(self, k):
        """
        Finds k words inserted into the trie most often.

        Parameters
        ----------
        k : int
            Number of most common words to be returned.

        Returns
        ----------
        list
            List of tuples.
            
            Each tuple entry consists of the word and its frequency.
            The entries are sorted by frequency.
        """
        
        sorted_count = {k: v for k, v in sorted(self.word_count.items(), key=lambda item: item[0])}
        sorted_count = {k: v for k, v in sorted(sorted_count.items(), key=lambda item: item[1], reverse = True)}

        lst_common = []
        
        for key, value in sorted_count.items():
            lst_common.append((key, value))
            k -=1
            if k == 0:
                break
        return lst_common
        
        
    def autocomplete(self, prefix):
        """
        Finds the most common word with the given prefix.

        Parameters
        ----------
        prefix : str
            The word part to be “autocompleted”.

        Returns
        ----------
        str
            The complete, most common word with the given prefix.
            
        Notes
        ----------
        The return value is equal to prefix if there is no valid word in the trie.
        The return value is also equal to prefix if prefix is the most common word.
        """

        words_with_pref = []
        
        #sorting the dictionary with counts again for this method
        sorted_count = {k: v for k, v in sorted(self.word_count.items(), key=lambda item: item[0])}
        sorted_count = {k: v for k, v in sorted(sorted_count.items(), key=lambda item: item[1], reverse = True)}
        
        sorted_words = sorted_count.keys()
        
        #picking words with the given prefix from the sorted list of words
        for word in sorted_words:
            if word.startswith(prefix):
                words_with_pref.append(word)
                
        #returning the first and most popular word with the given prefix
        the_word = words_with_pref[0]
        return the_word    
        

In [15]:
from requests import get
bad_chars = [';', ',', '.', '?', '!', '1', '2', '3', '4',
             '5', '6', '7', '8', '9', '0', '_', '[', ']']

SH_full = get('http://bit.ly/CS110-Shakespeare').text
SH_just_text = ''.join(c for c in SH_full if c not in bad_chars)
SH_without_newlines = ''.join(c if (c not in ['\n', '\r', '\t']) else " " for c in SH_just_text)
SH_just_words = [word for word in SH_without_newlines.split(" ") if word != ""]

SH_trie = Trie(SH_just_words)

assert SH_trie.autocomplete('hist') == 'history'
assert SH_trie.autocomplete('en') == 'enter'
assert SH_trie.autocomplete('cae') == 'caesar'
assert SH_trie.autocomplete('gen') == 'gentleman'
assert SH_trie.autocomplete('pen') == 'pen'
assert SH_trie.autocomplete('tho') == 'thou'
assert SH_trie.autocomplete('pent') == 'pentapolis'
assert SH_trie.autocomplete('petr') == 'petruchio'

### Additional tests

I'll use the Ukrainian novel again. Since we saw "так" (or "this way"/"yes") among the most popular words in the novel and there were no more popular words that would start with "та" we can expect the autocomplete to return "так"  if we input "та." 

In [16]:
trie = Trie(text)
trie.autocomplete('та') 

'так'

Yay, the output is the one that we expected. Out of curiousity let's see whether the code will be able to autocomplete numbers. 

In [17]:
trie = Trie(text)
trie.autocomplete('1') 

'1908'

It works for numbers too and return year 1908 as the most popular number that starts with 1. Probably just the year of publication. 

Let's also test it for the very first text provided:

In [18]:
trie = Trie(wordbank)
trie.autocomplete('l') 

'lantar'

The method I developed works! However, it is using the dictionary created as the tree is built that has all the words in the text as keys and the numbers of times those words are mentioned as values. I am using the whole list of words to find a specific subset that starts with a given prefix. Therefore, I have to go through all the words, which doesn't sound super efficient. The action of finding all the possible suggestion has the complexity of O(n) because of that. However, I can try utilizing the properties of the trie trees to improve this process at least a bit. We can traverse all the 

In [19]:
class Node:
    """
    This class represents one node of a trie tree.
    """

    def __init__(self, char):
        
        self.char = char
        self.children = []
        self.children_str = []
        self.is_end = False
        self.counter = 0
        
        #setting the node initially to an empty string
        self.str = ""
        
        
class Trie:
    """
    This class represents the entirety of a trie tree.

    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    """
    
    def __init__(self, word_list = None):
        """
        Creates the Trie instance, inserts initial words if provided.
        
        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
              
        self.root = Node("")
        self.word_count = {}
        
        if word_list:
            for word in word_list:
                word = word.lower()
                self.insert(word)
        
    
    def insert(self, word):
        """
        Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.
        """
        
        node = self.root 
        
        for char in word:
            found_in_child = False
            for child_node in node.children:
                if child_node.char == char:
                    node = child_node
                    found_in_child = True
        
            if not found_in_child:
                new_node = Node(char)
                node.children.append(new_node)
                node.children_str.append(char)
                node = new_node
        
        node.is_end = True
        node.counter += 1
        node.str = word
        self.word_count[word] = node.counter

        
    def lookup(self, word):
        """
        Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
        """

        node = self.root
        
        if not node.children:
            return False
        
        word = word.lower()
            
        for char in word:
            char_not_found = True
            for child in node.children:
                if child.char == char:
                    char_not_found = False
                    node = child
                    break
        
            if char_not_found:
                return False
        
        return node.is_end
    
    
    def k_most_common(self, k):
        """
        Finds k words inserted into the trie most often.

        Parameters
        ----------
        k : int
            Number of most common words to be returned.

        Returns
        ----------
        list
            List of tuples.
            
            Each tuple entry consists of the word and its frequency.
            The entries are sorted by frequency.
        """
        sorted_count = {k: v for k, v in sorted(self.word_count.items(), key=lambda item: item[0])}
        sorted_count = {k: v for k, v in sorted(sorted_count.items(), key=lambda item: item[1], reverse = True)}

        lst_common = []
        
        for key, value in sorted_count.items():
            lst_common.append((key, value))
            k -=1
            if k == 0:
                break
        return lst_common
    
    
    def suggestions(self, node, suggestions = None, level=0):
        '''
        Finds the all the words that start with the given prefix in the text

        Parameters
        ----------
        node : Node instance
            the ending node of a given prefix
        
        suggestions: list
            the list of the words with the prefix that is recursively increased

        Returns
        ----------
        suggestions: list
            list with the suggestions
        '''
        
        if suggestions is None:
            suggestions = []
            
        #Recursively traverse the trie starting from where given prefix ends
        ##and finding all the words that have the prefix
 
        for a in node.children:
            self.suggestions(a, suggestions, level+1)
        
        if node.is_end:
            suggestions.append(node.str)
            
        #print(" "*level, suggestions)
           
        return suggestions
        
        
    def autocomplete(self, prefix):
        """
        Finds the most common word with the given prefix.

        Parameters
        ----------
        prefix : str
            The word part to be “autocompleted”.

        Returns
        ----------
        str
            The complete, most common word with the given prefix.
        """
        
        
        #finding the node that is the end of the given prefix
        node = self.root 
        for a in prefix:
            if a not in node.children_str:
                break
            node = node.children[node.children_str.index(a)]
 
        # If there are no letters to continue the prefix returning prefix itself
        if not node.children:
            return prefix
        
        #finding the list of all the possible suggestions using suggestions method
        final = self.suggestions(node, suggestions = None)
        #print(final)
        #assigning the number of times the suggestions are mentioned to the words
        final_count = {}
        for suggestion in final:
            final_count[suggestion] = self.word_count[suggestion]
            
        #print(f"{final_count=}")
        
        #sorting solely suggestions alphabetically and then by the count
        sorted_count = {k: v for k, v in sorted(final_count.items(), key=lambda item: item[0])}
        sorted_count = {k: v for k, v in sorted(sorted_count.items(), key=lambda item: item[1], reverse = True)}
        
        #extracting the list of suggestions from the sorted dictionary 
        sorted_suggestions = list(sorted_count.keys())
        
        #returning the most popular suggestion
        #print(sorted_suggestions[0])
        return sorted_suggestions[0]

In [20]:
from requests import get
bad_chars = [';', ',', '.', '?', '!', '1', '2', '3', '4',
             '5', '6', '7', '8', '9', '0', '_', '[', ']']

SH_full = get('http://bit.ly/CS110-Shakespeare').text
SH_just_text = ''.join(c for c in SH_full if c not in bad_chars)
SH_without_newlines = ''.join(c if (c not in ['\n', '\r', '\t']) else " " for c in SH_just_text)
SH_just_words = [word for word in SH_without_newlines.split(" ") if word != ""]

SH_trie = Trie(SH_just_words)

In [21]:
assert SH_trie.autocomplete('hist') == 'history'
assert SH_trie.autocomplete('en') == 'enter'
assert SH_trie.autocomplete('cae') == 'caesar'
assert SH_trie.autocomplete('gen') == 'gentleman'
assert SH_trie.autocomplete('pen') == 'pen'
assert SH_trie.autocomplete('tho') == 'thou'
assert SH_trie.autocomplete('pent') == 'pentapolis'
assert SH_trie.autocomplete('petr') == 'petruchio'

Let's run the same three tests with this code to check whether it works as well.

In [22]:
trie = Trie(text)
trie.autocomplete('та') 

'так'

In [23]:
trie = Trie(text)
trie.autocomplete('1') 

'1908'

In [24]:
trie = Trie(wordbank)
trie.autocomplete('l') 

'lantar'

Yay! It works and returns the same suggestions as the previous a bit less efficient version. 